### Zadanie 1: Przygotowanie danych i podstawowa wizualizacja
Wygeneruj dane lub użyj dostępnej biblioteki (np. plotly.data).
Na podstawie tych danych stwórz aplikację Dash z jednym wykresem, który wyświetla:
gdpPercap (na osi Y) dla wybranych kontynentów (np. Azja, Ameryka) \
Elementy dodatkowe:
* dcc.Dropdown do wyboru kontynentu
* dcc.Graph do wyświetlenia wykresu

In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.gapminder()

continents = df['continent'].unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Wzrost PKB per capita wg kontynentu"),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': k, 'value': k} for k in continents],
        value='Asia',
        clearable=False
    ),
    dcc.Graph(id='gdp-plot')
])

@app.callback(
    Output('gdp-plot', 'figure'),
    Input('continent-dropdown', 'value')
)
def update_figure(chosen_continent):
    df_filtered = df[df['continent'] == chosen_continent]
    fig = px.line(df_filtered,
                  x='year',
                  y='gdpPercap',
                  color='country',
                  title=f'PKB per capita w {chosen_continent}')
    return fig


# Uruchom aplikację
if __name__ == '__main__':
    app.run(debug=True)

## Zadanie 2: Interaktywność — filtracja danych
Dodaj dodatkowe filtry:
* Suwak (dcc.Slider) do wyboru zakresu lat
* Dropdown do wyboru kraju lub kontynentu

Wykorzystaj callbacky, aby po zmianie filtrów automatycznie odświeżał się wykres.
Dodaj funkcję, która:
* Filtruje dane na podstawie wyboru
* Tworzy wykres słupkowy lub liniowy

In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

df = px.data.gapminder()
years = sorted(df['year'].unique())
continents = df['continent'].unique()
countries = df['country'].unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("PKB per capita — interaktywna wizualizacja"),

    html.Label("Wybierz kontynent lub kraj:"),
    dcc.Dropdown(
        id='region-dropdown',
        options=(
            [{'label': f"[Kontynent] {k}", 'value': k} for k in continents] +
            [{'label': f"[Kraj] {k}", 'value': k} for k in countries]
        ),
        value='Asia',
        clearable=False
    ),

    html.Label("Wybierz rok:"),
    dcc.RangeSlider(
        id='rok-rangeslider',
        min=min(years),
        max=max(years),
        value=[min(years), max(years)],
        marks={int(year): str(year) for year in years},
        step=None
    ),

    html.Label("Typ wykresu:"),
    dcc.Dropdown(
        id='plot-type',
        options=[
            {'label': 'Wykres liniowy', 'value': 'line'},
            {'label': 'Wykres słupkowy', 'value': 'bar'}
        ],
        value='line',
        clearable=False
    ),

    dcc.Graph(id='pkb-plot')
])

@app.callback(
    Output('pkb-plot', 'figure'),
    Input('region-dropdown', 'value'),
    Input('rok-rangeslider', 'value'),
    Input('plot-type', 'value')
)
def update_figure(chosen, year_range, plot_type):
    year_start, year_end = year_range

    if chosen in continents:
        dff = df[(df['continent'] == chosen) &
                 (df['year'] >= year_start) &
                 (df['year'] <= year_end)]
        title = f'PKB per capita — {chosen}, lata {year_start}–{year_end}'
        color = 'country'
    else:
        dff = df[(df['country'] == chosen) &
                 (df['year'] >= year_start) &
                 (df['year'] <= year_end)]
        title = f'PKB per capita — {chosen}, lata {year_start}–{year_end}'
        color = None

    if plot_type == 'line':
        fig = px.line(dff, x='year', y='gdpPercap', color=color, title=title)
    else:
        fig = px.bar(dff, x='year' if color is None else 'country',
                     y='gdpPercap', color=color, title=title)

    return fig

if __name__ == '__main__':
    app.run(debug=True)



## Zadanie 3: Rozbudowa dashboardu — wiele wykresów i elementów interaktywnych

Dodaj dwa wykresy:
* Jeden pokazujący PKB (gdpPercap) w zależności od roku
* Drugi pokazujący długość życia (lifeExp)

Dodaj filtry:
* Dropdown do wyboru kontynentu
* Suwak lat
* Użyj callbacków, które będą aktualizować oba wykresy jednocześnie w zależności od wybranych filtrów

In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Load dataset
df = px.data.gapminder()
continents = df['continent'].unique()
years = sorted(df['year'].unique())

# Init app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H2("GDP per Capita and Life Expectancy Over Time"),

    html.Label("Select Continent:"),
    dcc.Dropdown(
        id='continent-dropdown',
        options=[{'label': c, 'value': c} for c in continents],
        value='Europe',
        clearable=False
    ),

    html.Label("Select Year Range:"),
    dcc.RangeSlider(
        id='year-slider',
        min=min(years),
        max=max(years),
        value=[min(years), max(years)],
        marks={int(year): str(year) for year in years},
        step=None
    ),

    html.Div([
        dcc.Graph(id='gdp-graph'),
        dcc.Graph(id='lifeexp-graph')
    ])
])

# Callback
@app.callback(
    Output('gdp-graph', 'figure'),
    Output('lifeexp-graph', 'figure'),
    Input('continent-dropdown', 'value'),
    Input('year-slider', 'value')
)
def update_graphs(selected_continent, selected_years):
    start_year, end_year = selected_years

    filtered_df = df[
        (df['continent'] == selected_continent) &
        (df['year'] >= start_year) &
        (df['year'] <= end_year)
    ]

    gdp_fig = px.line(
        filtered_df, x='year', y='gdpPercap', color='country',
        title=f'GDP per Capita in {selected_continent} ({start_year}–{end_year})'
    )

    lifeexp_fig = px.line(
        filtered_df, x='year', y='lifeExp', color='country',
        title=f'Life Expectancy in {selected_continent} ({start_year}–{end_year})'
    )

    return gdp_fig, lifeexp_fig

# Run server
if __name__ == '__main__':
    app.run(debug=True)



## Zadanie 4: Wizualizacja własnych danych / API

Przygotuj własny plik CSV lub API zwracające przykładowe dane (np. lista sprzedaży, statystyki
sportowe, wyniki ankiet)
* Załaduj dane do Dash przy starcie aplikacji
* Wyświetl tabelę danych (dash_table.DataTable) oraz wykres słupkowy

In [7]:
import os
import json
import kaggle

with open('kaggle.json', 'r') as f:
    kaggle_creds = json.load(f)

os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']


kaggle.api.authenticate()
kaggle.api.dataset_download_files('jayaantanaath/student-habits-vs-academic-performance', path='C:\\Users\\dawid\\PycharmProjects\\WizualizacjaDanych\\data', unzip=True)


OSError: Could not find kaggle.json. Make sure it's located in C:\Users\dawid\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [37]:
import dash
from dash import html, dcc, dash_table
import pandas as pd
import plotly.express as px

df = pd.read_csv("data/student_habits_performance.csv")

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Student Habits vs Academic Performance"),

    html.Label("Filter by Gender:"),
    dcc.Dropdown(
        id='gender-filter',
        options=[{"label": g, "value": g} for g in df["gender"].unique()],
        value=None,
        placeholder="Select gender (optional)",
        clearable=True
    ),

    html.Br(),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'}
    ),

    html.Br(),
    dcc.Graph(id='bar-graph')
])

@app.callback(
    [dash.dependencies.Output('data-table', 'data'),
     dash.dependencies.Output('bar-graph', 'figure')],
    [dash.dependencies.Input('gender-filter', 'value')]
)
def update_output(selected_gender):
    dff = df.copy()
    if selected_gender:
        dff = dff[dff['gender'] == selected_gender]

    fig = px.bar(
        dff,
        x='study_hours_per_day',
        y='exam_score',
        color='gender',
        title='Exam Score vs Study Hours',
        labels={'study_hours_per_day': 'Study Hours per Day', 'exam_score': 'Exam Score'}
    )

    return dff.to_dict('records'), fig

if __name__ == '__main__':
    app.run(debug=True)



## Zadanie 5: Aktualizacja danych i odświeżanie w czasie rzeczywistym

Dodaj komponent dcc.Interval, który co 1-2 sekundy będzie generować nowe dane (np. losowe wartości lub pobierać z API). \
Callback na podstawie tego interwału będzie odświeżał wykres lub licznik.

In [39]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import random
import datetime

app = dash.Dash(__name__)

data = pd.DataFrame(columns=["time", "value"])

app.layout = html.Div([
    html.H2("Wykres aktualizowany w czasie rzeczywistym"),

    dcc.Graph(id='live-graph'),

    dcc.Interval(
        id='interval-component',
        interval=1000,
        n_intervals=0
    )
])


@app.callback(
    Output('live-graph', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_graph(n):
    global data

    now = datetime.datetime.now().strftime("%H:%M:%S")
    new_value = random.randint(0, 100)

    if len(data) == 0 or data.iloc[-1]['time'] != now:
        new_row = pd.DataFrame({"time": [now], "value": [new_value]})
        data = pd.concat([data, new_row], ignore_index=True)

    data = data.tail(20)

    fig = px.line(data, x="time", y="value", title="Losowe dane w czasie rzeczywistym")
    fig.update_layout(uirevision=True)

    return fig


if __name__ == '__main__':
    app.run(debug=True)



## Zadanie 6: Ekspozycja na dużą ilość danych – wydajność i limitacje (60 min)

* Utwórz dużą próbkę danych (np. 100 000 punktów)
* Wyświetl je na wykresie
* Spróbuj zoptymalizować: podziel dane na segmenty, użyć agregacji, lub ogranicz liczbę punktów (np. tylko co 10)

In [46]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np

np.random.seed(42)
df = pd.DataFrame({
    "x": np.arange(100_000),
    "y": np.random.normal(loc=0, scale=1, size=100_000)
})

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Wykres dla dużej ilości danych (100 000 punktów)"),

    html.Label("Krok filtrowania (im większy, tym mniej punktów):"),
    dcc.Slider(
        id='step-slider',
        min=1,
        max=100,
        value=10,
        marks={1: '1', 10: '10', 50: '50', 100: '100'},
        step=1
    ),

    dcc.Graph(id='big-data-graph')
])

@app.callback(
    Output('big-data-graph', 'figure'),
    Input('step-slider', 'value')
)
def update_graph(step):
    # Filtrowanie co N-tego punktu dla optymalizacji
    filtered_df = df.iloc[::step]
    fig = px.line(filtered_df, x="x", y="y", title=f"Co {step}-ty punkt z 100 000")
    fig.update_layout(uirevision=True)
    return fig

if __name__ == '__main__':
    app.run(debug=True)



### Zadanie 7: Export i podzielenie danych do PDF, PNG

Dodaj funkcję, która po kliknięciu przycisku zapisze aktualny wykres do pliku PNG lub PDF.

In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.io as pio
from datetime import datetime

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H2("Wykres z możliwością eksportu"),

    dcc.Graph(id='export-graph'),

    html.Div([
        dcc.Dropdown(
            id='export-format',
            options=[
                {'label': 'PNG', 'value': 'png'},
                {'label': 'PDF', 'value': 'pdf'}
            ],
            value='png',
            style={'width': '100px', 'display': 'inline-block', 'marginRight': '10px'}
        ),
        html.Button('Export', id='export-button', n_clicks=0)
    ]),

    dcc.Download(id='download-plot')
])


@app.callback(
    Output('export-graph', 'figure')
)
def update_graph():
    fig = np.sin(1)
    return fig


@app.callback(
    Output('download-plot', 'data'),
    Input('export-button', 'n_clicks'),
    Input('export-format', 'value'),
    Input('export-graph', 'figure'),
    prevent_initial_call=True
)
def export_plot(n_clicks, format_type, figure):
    if n_clicks > 0:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"plot_{timestamp}.{format_type}"
        return dict(content=pio.to_image(figure, format=format_type), filename=filename)
    return None


if __name__ == '__main__':
    app.run(debug=True)